# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd


In [2]:
# Your code here; create the connection
# Create a connection to the database
conn = sqlite3.connect('data.sqlite')

# Create a cursor object
cur = conn.cursor()


## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
# Your code here
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect('data.sqlite')

# Define the query
query = '''
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
);
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the results
print(df)

# Close the connection
conn.close()


   customerNumber contactLastName contactFirstName
0             141          Freyre           Diego 


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect('data.sqlite')

# Define the query
query = '''
SELECT
    p.productName,
    COUNT(od.orderNumber) AS totalOrders,
    SUM(od.quantityOrdered) AS totalItemsSold
FROM
    products p
JOIN
    orderdetails od ON p.productCode = od.productCode
GROUP BY
    p.productName
ORDER BY
    totalItemsSold DESC;
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the results
print(df)

# Close the connection
conn.close()


                                 productName  totalOrders  totalItemsSold
0                1992 Ferrari 360 Spider red           53            1808
1                       1937 Lincoln Berline           28            1111
2                  American Airlines: MD-11S           28            1085
3    1941 Chevrolet Special Deluxe Cabriolet           28            1076
4               1930 Buick Marquette Phaeton           28            1074
..                                       ...          ...             ...
104             1999 Indy 500 Monte Carlo SS           25             855
105                       1911 Ford Town Car           25             832
106         1936 Mercedes Benz 500k Roadster           25             824
107               1970 Chevy Chevelle SS 454           25             803
108                    1957 Ford Thunderbird           24             767

[109 rows x 3 columns]


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [5]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect('data.sqlite')

# Define the query
query = '''
SELECT DISTINCT city
FROM customers;
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the results
print(df)

# Close the connection
conn.close()


                city
0             Nantes
1          Las Vegas
2          Melbourne
3            Stavern
4         San Rafael
..               ...
91          Mannheim
92  Saint Petersburg
93           Herzlia
94           Sevilla
95          Brisbane

[96 rows x 1 columns]


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [6]:
# Your code here
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect('data.sqlite')

# Define the query
query = '''
SELECT DISTINCT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    o.city,
    e.officeCode
FROM
    employees e
JOIN
    offices o ON e.officeCode = o.officeCode
JOIN
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN
    orders ord ON c.customerNumber = ord.customerNumber
JOIN
    orderdetails od ON ord.orderNumber = od.orderNumber
WHERE
    od.productCode IN (
        SELECT
            productCode
        FROM
            orderdetails
        GROUP BY
            productCode
        HAVING
            COUNT(DISTINCT orderNumber) < 20
    );
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the results
print(df)

# Close the connection
conn.close()


Empty DataFrame
Columns: [employeeNumber, firstName, lastName, city, officeCode]
Index: []


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [7]:
# Your code here
import sqlite3
import pandas as pd

# Create a connection to the database
conn = sqlite3.connect('data.sqlite')

# Define the query
query = '''
SELECT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    COUNT(c.customerNumber) AS numberOfCustomers
FROM
    employees e
JOIN
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY
    e.employeeNumber
HAVING
    AVG(c.creditLimit) > 15000;
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the results
print(df)

# Close the connection
conn.close()


    employeeNumber firstName   lastName  numberOfCustomers
0             1165    Leslie   Jennings                  6
1             1166    Leslie   Thompson                  6
2             1188     Julie   Firrelli                  6
3             1216     Steve  Patterson                  6
4             1286  Foon Yue      Tseng                  7
5             1323    George     Vanauf                  8
6             1337      Loui     Bondur                  6
7             1370    Gerard  Hernandez                  7
8             1401    Pamela   Castillo                 10
9             1501     Larry       Bott                  8
10            1504     Barry      Jones                  9
11            1611      Andy     Fixter                  5
12            1612     Peter      Marsh                  5
13            1621      Mami      Nishi                  5
14            1702    Martin     Gerard                  6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!